### **Transformers y atención como bloque universal** 

Este cuaderno cubre, con implementaciones **desde cero en PyTorch** :

- **Self-attention (autoatención)**: consultas (**Q**), claves (**K**) y valores (**V**), incluyendo su **complejidad cuadrática**.
- **Multi-head attention**, **LayerNorm** y **conexiones residuales**.
- **Arquitectura encoder/decoder** y variantes (**encoder-only**, **decoder-only para LLM**).
- **Codificación posicional** y variantes modernas (**sin/cos**, **learned**, **RoPE**, **ALiBi**).

> Enfoque: cuaderno didáctico, ejecutable en CPU, usando un corpus pequeño y un tokenizador simple en Python (sin `torchtext`).


#### **Estructura del cuaderno**

1. Preparación del entorno y tokenización simple (sin `torchtext`).
2. Codificaciones posicionales: sinusoidal, learned, **RoPE** y **ALiBi**.
3. Self-attention (Q, K, V), máscara causal y complejidad $O(T^2)$.
4. Multi-head attention + residual + LayerNorm + FFN (bloque Transformer).
5. Encoder vs. Decoder y variantes (encoder-only / decoder-only).
6. Mini-entrenamiento autoregresivo (decoder-only) con `AdamW`, warmup y clipping.
7. Decodificación básica (temperatura, top-k, top-p).


##### **Cómo trabajar este cuaderno en clase**

- Las celdas de implementación están listas para ejecutar.
- Las secciones **Actividad guiada** son para discusión y respuesta escrita.
- Las celdas **Respuesta del estudiante** se pueden editar directamente en clase o como tarea.
- Las celdas **Extensión opcional** permiten experimentar sin modificar el código base.

In [ ]:

import math
import random
from dataclasses import dataclass
from collections import Counter
from typing import List, Dict, Tuple

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import matplotlib.pyplot as plt

# Reproducibilidad
SEED = 42
random.seed(SEED)
torch.manual_seed(SEED)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Dispositivo:", device)


#### **1. Tokenización simple y vocabulario**

Para mantener el cuaderno autocontenido, usaremos un tokenizador básico por espacios y un vocabulario propio.

Esto es suficiente para ilustrar el pipeline de un Transformer:

**texto -> tokens -> ids -> embeddings -> codificación posicional -> bloques Transformer**


In [ ]:
corpus = [
    "the llama learns quickly",
    "the llama runs fast",
    "the dog runs fast",
    "the dog barks loudly",
    "the horse runs fast",
    "the horse eats hay",
    "the llama eats hay",
    "attention is a universal block",
    "transformers use self attention",
    "decoder only models predict next token",
    "encoder decoder models use cross attention",
]

def basic_tokenize(text: str) -> List[str]:
    return text.lower().strip().split()

class SimpleVocab:
    def __init__(self, token_lists: List[List[str]], min_freq: int = 1):
        specials = ["<pad>", "<bos>", "<eos>", "<unk>"]
        counter = Counter(tok for toks in token_lists for tok in toks)
        self.itos = specials[:]
        for tok, freq in counter.items():
            if freq >= min_freq and tok not in specials:
                self.itos.append(tok)
        self.stoi = {tok: i for i, tok in enumerate(self.itos)}
        self.pad_id = self.stoi["<pad>"]
        self.bos_id = self.stoi["<bos>"]
        self.eos_id = self.stoi["<eos>"]
        self.unk_id = self.stoi["<unk>"]

    def encode(self, text: str, add_bos: bool = True, add_eos: bool = True) -> List[int]:
        ids = []
        if add_bos:
            ids.append(self.bos_id)
        ids.extend(self.stoi.get(tok, self.unk_id) for tok in basic_tokenize(text))
        if add_eos:
            ids.append(self.eos_id)
        return ids

    def decode(self, ids: List[int]) -> str:
        toks = []
        for idx in ids:
            tok = self.itos[idx]
            if tok in {"<pad>", "<bos>", "<eos>"}:
                continue
            toks.append(tok)
        return " ".join(toks)

token_lists = [basic_tokenize(x) for x in corpus]
vocab = SimpleVocab(token_lists)
vocab_size = len(vocab.itos)

print("Tamaño del vocabulario:", vocab_size)
print("Muestra del vocabulario:", vocab.itos[:20])

sample_text = "the llama runs fast"
sample_ids = vocab.encode(sample_text)
print("Texto:", sample_text)
print("Codificado:", sample_ids)
print("Decodificado:", vocab.decode(sample_ids))


#### **Actividad guiada 1 - Pipeline de entrada (texto, no código)**

Responde de forma argumentada:

1. Explica el rol de cada etapa del pipeline:
   **texto -> tokens -> ids -> embeddings -> codificación posicional -> bloque Transformer**.  
   Indica qué información se conserva y qué información se transforma en cada paso.

2. ¿Por qué usar tokens especiales como `<bos>`, `<eos>`, `<pad>` y `<unk>`?  
   Describe un caso concreto donde cada uno sea necesario.

3. Si el vocabulario se construye con un corpus pequeño, ¿qué limitaciones aparecen al generar texto o evaluar ejemplos nuevos?

4. Compara tokenización por espacios vs. subword tokenization (BPE/WordPiece/SentencePiece).  
   ¿Qué problema resuelve la tokenización por subpalabras?

##### **Respuesta del estudiante - Actividad 1**

Escribe aquí tus respuestas (texto):
- **1. Pipeline:**  
- **2. Tokens especiales:**  
- **3. Limitaciones del vocabulario pequeño:**  
- **4. Comparación con subwords:**  

In [ ]:
# Extensión opcional (estudiante)
# Prueba agregar nuevas oraciones al corpus y vuelve a construir el vocabulario.
# Luego compara:
# - tamaño del vocabulario
# - presencia de tokens <unk>
#
# Sugerencia:
# corpus_ext = corpus + ["...","..."]
# token_lists_ext = [basic_tokenize(x) for x in corpus_ext]
# vocab_ext = SimpleVocab(token_lists_ext)
# print("Nuevo tamaño:", len(vocab_ext.itos))

#### **2. Codificación posicional y variantes**

##### **2.1 Sinusoidal (absoluto)**
Es la codificación posicional clásica del Transformer original. No introduce parámetros extra y permite codificar posición con funciones seno/coseno.

##### **2.2 Learned positional embeddings (absoluto)**
Se aprende un vector por posición. Suele funcionar bien, aunque extrapola peor fuera del rango visto durante entrenamiento.

##### **2.3 RoPE (Rotary Positional Embedding)**
Rota pares de dimensiones de **Q** y **K** según la posición, inyectando posición de manera **relativa** en la atención. Es muy usado en LLMs modernos.

##### **2.4 ALiBi**
Añade un **sesgo lineal** a los logits de atención según la distancia entre posiciones. Favorece relaciones locales y suele escalar bien a contextos largos.


In [ ]:

class SinusoidalPositionalEncoding(nn.Module):
    def __init__(self, d_model: int, max_len: int = 512):
        super().__init__()
        pe = torch.zeros(max_len, d_model)
        pos = torch.arange(0, max_len, dtype=torch.float32).unsqueeze(1)
        div = torch.exp(torch.arange(0, d_model, 2, dtype=torch.float32) * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(pos * div)
        pe[:, 1::2] = torch.cos(pos * div)
        self.register_buffer("pe", pe.unsqueeze(0))  # [1, T, D]

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        # x: [B, T, D]
        T = x.size(1)
        return x + self.pe[:, :T, :]

class LearnedPositionalEncoding(nn.Module):
    def __init__(self, d_model: int, max_len: int = 512):
        super().__init__()
        self.pos_emb = nn.Embedding(max_len, d_model)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        B, T, D = x.shape
        pos = torch.arange(T, device=x.device).unsqueeze(0).expand(B, T)
        return x + self.pos_emb(pos)

# Comparación visual
d_model = 32
T_vis = 64
dummy = torch.zeros(1, T_vis, d_model)

sin_pe = SinusoidalPositionalEncoding(d_model, max_len=256)
learned_pe = LearnedPositionalEncoding(d_model, max_len=256)

with torch.no_grad():
    sin_out = sin_pe(dummy)[0].cpu()
    learned_out = learned_pe(dummy)[0].cpu()

plt.figure(figsize=(10, 3))
plt.imshow(sin_out.T, aspect="auto")
plt.title("Codificación posicional sinusoidal (T x D)")
plt.xlabel("Posición")
plt.ylabel("Dimensión")
plt.colorbar()
plt.show()

plt.figure(figsize=(10, 3))
plt.imshow(learned_out.T, aspect="auto")
plt.title("Embeddings posicionales aprendidos (T x D)")
plt.xlabel("Posición")
plt.ylabel("Dimensión")
plt.colorbar()
plt.show()



##### **2.5 Implementación de RoPE y ALiBi (bloques reutilizables)**

A continuación implementamos funciones reutilizables para:

- aplicar **RoPE** sobre tensores `[..., T, D]` (con `D` par),
- construir el sesgo de atención de **ALiBi** por cabecera.


In [ ]:
def _rotate_half(x: torch.Tensor) -> torch.Tensor:
    # x: [..., D], D debe ser par
    x1 = x[..., ::2]
    x2 = x[..., 1::2]
    out = torch.stack((-x2, x1), dim=-1).flatten(-2)
    return out

def build_rope_cache(seq_len: int, dim: int, device=None, base: float = 10000.0):
    assert dim % 2 == 0, "RoPE requiere dimensión par por cabecera"
    pos = torch.arange(seq_len, dtype=torch.float32, device=device).unsqueeze(1)      # [T,1]
    freq = torch.arange(0, dim, 2, dtype=torch.float32, device=device)                # [D/2]
    inv_freq = 1.0 / (base ** (freq / dim))                                            # [D/2]
    angles = pos * inv_freq.unsqueeze(0)                                               # [T,D/2]
    # Expandir a dimensión completa intercalando
    cos = torch.repeat_interleave(torch.cos(angles), repeats=2, dim=-1)                # [T,D]
    sin = torch.repeat_interleave(torch.sin(angles), repeats=2, dim=-1)                # [T,D]
    return cos, sin

def apply_rope(x: torch.Tensor, cos: torch.Tensor, sin: torch.Tensor) -> torch.Tensor:
    # x: [B, H, T, D]
    # cos/sin: [T, D]
    cos = cos.unsqueeze(0).unsqueeze(0)  # [1,1,T,D]
    sin = sin.unsqueeze(0).unsqueeze(0)
    return (x * cos) + (_rotate_half(x) * sin)

def build_alibi_bias(num_heads: int, seq_len: int, device=None) -> torch.Tensor:
    # Pendientes geométricas simples (versión didáctica)
    slopes = torch.tensor([2 ** (-(i + 1) / num_heads) for i in range(num_heads)],
                          dtype=torch.float32, device=device)  # [H]
    i = torch.arange(seq_len, device=device)
    j = torch.arange(seq_len, device=device)
    dist = (i[None, :] - j[:, None]).abs().float()   # [T,T]
    # Forma del sesgo [1, H, T, T]
    bias = -slopes.view(1, num_heads, 1, 1) * dist.view(1, 1, seq_len, seq_len)
    return bias

# Tensores de demostración
B, H, T, Dh = 1, 2, 8, 8
q = torch.randn(B, H, T, Dh)
k = torch.randn(B, H, T, Dh)

cos, sin = build_rope_cache(seq_len=T, dim=Dh, device=q.device)
q_rope = apply_rope(q, cos, sin)
k_rope = apply_rope(k, cos, sin)

alibi = build_alibi_bias(num_heads=H, seq_len=T, device=q.device)

print("Forma de q:", q.shape)
print("Forma de q_rope:", q_rope.shape)
print("Forma del sesgo ALiBi:", alibi.shape)

plt.figure(figsize=(4, 3))
plt.imshow(alibi[0, 0].cpu(), aspect="auto")
plt.title("Sesgo ALiBi (cabecera 0)")
plt.xlabel("Posición clave")
plt.ylabel("Posición consulta")
plt.colorbar()
plt.show()


##### **Actividad guiada 2 - Posición en Transformers (texto, no código)**

1. Explica con tus palabras por qué un mecanismo de atención puro **no conoce el orden** de los tokens.

2. Compara conceptualmente:
   - **Sinusoidal** (absoluto)
   - **Learned positional embeddings** (absoluto)
   - **RoPE** (posición incorporada en Q/K)
   - **ALiBi** (sesgo en logits)

   Para cada uno, menciona:
   - dónde se aplica,
   - si agrega parámetros,
   - una ventaja y una posible limitación.

3. ¿Por qué RoPE y ALiBi suelen aparecer en modelos modernos orientados a contextos largos?

4. Si un modelo fue entrenado con longitud máxima 512, ¿qué problemas podrían aparecer al usar secuencias mucho más largas?

##### **Respuesta del estudiante- Actividad 2**

Completa esta comparación (texto libre o tabla):

- **Sinusoidal:**  
- **Learned:**  
- **RoPE:**  
- **ALiBi:**  

**Conclusión personal:**  

In [ ]:
# Extensión opcional (estudiante)
# Grafica el sesgo ALiBi para distintas cabeceras y comenta cómo cambia.
# Sugerencia:
# bias = build_alibi_bias(num_heads=4, seq_len=16)
# plt.imshow(bias[0, 0].cpu()); plt.colorbar(); plt.show()

#### **3. Self-attention: Q, K, V y complejidad $O(T^2)$**

Recordatorio (por cabecera):
$$
\text{Atencion}(Q,K,V)=\text{softmax}\left(\frac{QK^\top}{\sqrt{d_k}} + \text{mask}\right)V
$$

- **Q (queries)**: qué busca cada token.
- **K (keys)**: cómo se indexa cada token.
- **V (values)**: contenido que se mezcla.
- La matriz de atención tiene tamaño **T×T**, por eso el costo escala como **$O(T^2)$** en secuencia.


In [ ]:
def make_causal_mask(T: int, device=None) -> torch.Tensor:
    # True significa "enmascarar"
    return torch.triu(torch.ones(T, T, device=device, dtype=torch.bool), diagonal=1)

def scaled_dot_product_attention(q, k, v, attn_mask=None, extra_bias=None, dropout_p=0.0, training=False):
    '''
    q, k, v: [B, H, T, D]
    attn_mask: [T, T] bool (True = enmascarado)
    extra_bias: [1, H, T, T] or broadcastable (e.g., ALiBi)
    '''
    d = q.size(-1)
    scores = (q @ k.transpose(-2, -1)) / math.sqrt(d)   # [B,H,T,T]
    if extra_bias is not None:
        scores = scores + extra_bias
    if attn_mask is not None:
        scores = scores.masked_fill(attn_mask.unsqueeze(0).unsqueeze(0), float('-inf'))
    attn = F.softmax(scores, dim=-1)
    if dropout_p > 0:
        attn = F.dropout(attn, p=dropout_p, training=training)
    out = attn @ v                                        # [B,H,T,D]
    return out, attn, scores


In [ ]:
# Demostración: una secuencia con embeddings aleatorios proyectados a Q/K/V
B, T, D_model = 1, 6, 32
H = 4
Dh = D_model // H

x = torch.randn(B, T, D_model)

Wq = nn.Linear(D_model, D_model, bias=False)
Wk = nn.Linear(D_model, D_model, bias=False)
Wv = nn.Linear(D_model, D_model, bias=False)

q = Wq(x).view(B, T, H, Dh).transpose(1, 2)  # [B,H,T,Dh]
k = Wk(x).view(B, T, H, Dh).transpose(1, 2)
v = Wv(x).view(B, T, H, Dh).transpose(1, 2)

causal_mask = make_causal_mask(T, device=x.device)

# Variante A: base
out_vanilla, attn_vanilla, _ = scaled_dot_product_attention(q, k, v, attn_mask=causal_mask)

# Variante B: con RoPE + ALiBi
cos, sin = build_rope_cache(seq_len=T, dim=Dh, device=x.device)
q_rope = apply_rope(q, cos, sin)
k_rope = apply_rope(k, cos, sin)
alibi_bias = build_alibi_bias(H, T, device=x.device)

out_modern, attn_modern, _ = scaled_dot_product_attention(
    q_rope, k_rope, v, attn_mask=causal_mask, extra_bias=alibi_bias
)

print("Forma de salida (base):", out_vanilla.shape)
print("Forma de atención:", attn_vanilla.shape)  # [B,H,T,T]

plt.figure(figsize=(5, 3))
plt.imshow(attn_vanilla[0, 0].detach().cpu(), aspect="auto")
plt.title("Pesos de self-attention (cabecera 0, causal)")
plt.xlabel("Posición clave")
plt.ylabel("Posición consulta")
plt.colorbar()
plt.show()


In [ ]:
# Intuición de complejidad: la matriz de logits de atención es T x T
def attention_score_matrix_size(T: int, H: int = 8):
    return H * T * T

Ts = [16, 32, 64, 128, 256, 512]
sizes = [attention_score_matrix_size(T, H=8) for T in Ts]

plt.figure(figsize=(6, 3))
plt.plot(Ts, sizes, marker="o")
plt.title("Escala cuadrática del tamaño de logits de atención (8 cabeceras)")
plt.xlabel("Longitud de secuencia T")
plt.ylabel("Elementos en logits de atención (H*T*T)")
plt.show()

# Pequeño benchmark de tiempo (apto para CPU)
def benchmark_attention(T=128, D_model=128, H=8, reps=20, use_rope=False, use_alibi=False):
    B = 4
    Dh = D_model // H
    x = torch.randn(B, T, D_model)
    Wq, Wk, Wv = nn.Linear(D_model, D_model, bias=False), nn.Linear(D_model, D_model, bias=False), nn.Linear(D_model, D_model, bias=False)
    q = Wq(x).view(B, T, H, Dh).transpose(1, 2)
    k = Wk(x).view(B, T, H, Dh).transpose(1, 2)
    v = Wv(x).view(B, T, H, Dh).transpose(1, 2)
    bias = None
    if use_rope:
        cos, sin = build_rope_cache(T, Dh, device=x.device)
        q = apply_rope(q, cos, sin)
        k = apply_rope(k, cos, sin)
    if use_alibi:
        bias = build_alibi_bias(H, T, device=x.device)
    mask = make_causal_mask(T)
    # calentamiento
    for _ in range(3):
        _ = scaled_dot_product_attention(q, k, v, attn_mask=mask, extra_bias=bias)
    start = torch.cuda.Event(enable_timing=True) if torch.cuda.is_available() else None
    end = torch.cuda.Event(enable_timing=True) if torch.cuda.is_available() else None
    if start is not None:
        torch.cuda.synchronize()
        start.record()
        for _ in range(reps):
            _ = scaled_dot_product_attention(q, k, v, attn_mask=mask, extra_bias=bias)
        end.record()
        torch.cuda.synchronize()
        ms = start.elapsed_time(end) / reps
    else:
        import time
        t0 = time.perf_counter()
        for _ in range(reps):
            _ = scaled_dot_product_attention(q, k, v, attn_mask=mask, extra_bias=bias)
        ms = (time.perf_counter() - t0) * 1000 / reps
    return ms

for T in [64, 128, 256]:
    ms = benchmark_attention(T=T, D_model=128, H=8, reps=10)
    print(f"T={T:>3} -> ~{ms:.2f} ms / forward (solo atención scaled-dot)")


##### **Actividad guiada 3 - Self-attention y complejidad**

1. Interpreta Q, K y V en términos de una búsqueda de información:
   - ¿qué representa una **query**?
   - ¿qué representa una **key**?
   - ¿qué representa un **value**?

2. Explica la función de la **máscara causal** en un modelo autoregresivo.
   ¿Qué error conceptual habría si el decoder pudiera ver tokens futuros durante entrenamiento?

3. La matriz de atención por cabecera tiene tamaño **T × T**.
   Explica por qué esto implica crecimiento cuadrático en memoria/cómputo y qué efecto tiene al pasar de $T=128$ a $T=1024$.

4. Observa el benchmark de tiempo:
   ¿por qué el tiempo real no crece exactamente como una curva matemática ideal, aunque la complejidad teórica sí sea cuadrática?

##### **Respuesta del estudiante - Actividad 3**

- **Q/K/V como búsqueda de información:**  
- **Máscara causal:**  
- **Complejidad $O(T^2)$:**  
- **Diferencia entre teoría y benchmark:**  

In [ ]:
# Extensión opcional (estudiante)
# Compara mapas de atención con y sin máscara causal.
# Puedes reutilizar q, k, v y llamar scaled_dot_product_attention(...)

#### **4. Multi-head attention + residual + LayerNorm + FFN (bloque Transformer)**

Ahora encapsulamos la atención en un módulo reusable:

- proyecciones **Q/K/V**
- **multi-head**
- opción de **RoPE** y/o **ALiBi**
- **dropout**
- **residual + LayerNorm**
- **FeedForward (MLP)**


In [ ]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model: int, n_heads: int, dropout: float = 0.1, use_rope: bool = False, use_alibi: bool = False):
        super().__init__()
        assert d_model % n_heads == 0
        self.d_model = d_model
        self.n_heads = n_heads
        self.d_head = d_model // n_heads
        self.use_rope = use_rope
        self.use_alibi = use_alibi

        self.q_proj = nn.Linear(d_model, d_model, bias=False)
        self.k_proj = nn.Linear(d_model, d_model, bias=False)
        self.v_proj = nn.Linear(d_model, d_model, bias=False)
        self.o_proj = nn.Linear(d_model, d_model, bias=False)
        self.dropout = nn.Dropout(dropout)

    def _split_heads(self, x):
        # x: [B,T,D] -> [B,H,T,Dh]
        B, T, D = x.shape
        return x.view(B, T, self.n_heads, self.d_head).transpose(1, 2)

    def _merge_heads(self, x):
        # x: [B,H,T,Dh] -> [B,T,D]
        B, H, T, Dh = x.shape
        return x.transpose(1, 2).contiguous().view(B, T, H * Dh)

    def forward(self, x_q, x_kv=None, attn_mask=None):
        '''
        x_q:  [B,Tq,D]
        x_kv: [B,Tk,D] (if None, self-attention)
        '''
        if x_kv is None:
            x_kv = x_q

        q = self._split_heads(self.q_proj(x_q))
        k = self._split_heads(self.k_proj(x_kv))
        v = self._split_heads(self.v_proj(x_kv))

        B, H, Tq, Dh = q.shape
        Tk = k.size(2)

        extra_bias = None
        # RoPE suele aplicarse cuando Tq == Tk (self-attention)
        if self.use_rope and Tq == Tk:
            cos, sin = build_rope_cache(seq_len=Tk, dim=Dh, device=q.device)
            q = apply_rope(q, cos, sin)
            k = apply_rope(k, cos, sin)

        if self.use_alibi:
            extra_bias = build_alibi_bias(H, Tk, device=q.device)[:, :, :Tq, :Tk]

        out, attn, _ = scaled_dot_product_attention(
            q, k, v,
            attn_mask=attn_mask,
            extra_bias=extra_bias,
            dropout_p=self.dropout.p,
            training=self.training
        )
        out = self._merge_heads(out)
        out = self.o_proj(out)
        return out, attn

class FeedForward(nn.Module):
    def __init__(self, d_model: int, d_ff: int, dropout: float = 0.1):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(d_model, d_ff),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(d_ff, d_model),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class TransformerBlock(nn.Module):
    '''Bloque Pre-Norm: LN -> MHA -> residual -> LN -> FFN -> residual'''
    def __init__(self, d_model: int, n_heads: int, d_ff: int, dropout: float = 0.1, use_rope=False, use_alibi=False):
        super().__init__()
        self.ln1 = nn.LayerNorm(d_model)
        self.attn = MultiHeadAttention(d_model, n_heads, dropout=dropout, use_rope=use_rope, use_alibi=use_alibi)
        self.ln2 = nn.LayerNorm(d_model)
        self.ffn = FeedForward(d_model, d_ff, dropout=dropout)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, attn_mask=None):
        h, attn = self.attn(self.ln1(x), attn_mask=attn_mask)
        x = x + self.dropout(h)    # residual
        x = x + self.ffn(self.ln2(x))  # residual
        return x, attn

# Demostración forward
B, T, D = 2, 10, 64
x_demo = torch.randn(B, T, D)
block = TransformerBlock(d_model=D, n_heads=4, d_ff=256, use_rope=True, use_alibi=True)
y_demo, attn_demo = block(x_demo, attn_mask=make_causal_mask(T))
print("Salida del bloque:", y_demo.shape, "Atención:", attn_demo.shape)


##### **Actividad guiada 4 - Bloque Transformer universal**

1. Explica por qué el bloque Transformer se considera **un bloque universal** reutilizable en encoder-only, decoder-only y encoder/decoder.

2. Describe la función de cada componente en el bloque:
   - Multi-head attention
   - Conexión residual
   - LayerNorm
   - FFN (MLP)

3. ¿Qué problema de entrenamiento ayudan a mitigar:
   - las residuales,
   - la normalización,
   - el dropout?

4. ¿Qué significa que el bloque implementado sea **Pre-Norm**?
   ¿En qué orden se aplican LN, atención y FFN?

##### **Respuesta del estudiante - Actividad 4**

- **Bloque universal:**  
- **Rol de cada componente:**  
- **Estabilidad del entrenamiento:**  
- **Pre-Norm:**  

#### **5. Encoder/Decoder y variantes (encoder-only, decoder-only)**

##### **Bloque encoder**
- Self-attention **no causal** (puede mirar toda la secuencia)
- Útil para **representaciones contextualizadas** (BERT, clasificación, MLM, etc.)

##### **Bloque decoder**
- Self-attention **causal** (máscara triangular)
- Opcionalmente **cross-attention (atención cruzada)** si hay un encoder (arquitectura encoder/decoder)
- Base de modelos **decoder-only** para LLM (next-token prediction)


In [ ]:
class EncoderBlock(nn.Module):
    def __init__(self, d_model, n_heads, d_ff, dropout=0.1):
        super().__init__()
        self.block = TransformerBlock(d_model, n_heads, d_ff, dropout=dropout, use_rope=False, use_alibi=False)

    def forward(self, x):
        # Self-attention completo, sin máscara causal
        return self.block(x, attn_mask=None)

class DecoderBlock(nn.Module):
    def __init__(self, d_model, n_heads, d_ff, dropout=0.1, use_rope=True, use_alibi=False):
        super().__init__()
        self.ln1 = nn.LayerNorm(d_model)
        self.self_attn = MultiHeadAttention(d_model, n_heads, dropout=dropout, use_rope=use_rope, use_alibi=use_alibi)

        self.ln_cross = nn.LayerNorm(d_model)
        self.cross_attn = MultiHeadAttention(d_model, n_heads, dropout=dropout, use_rope=False, use_alibi=False)

        self.ln2 = nn.LayerNorm(d_model)
        self.ffn = FeedForward(d_model, d_ff, dropout=dropout)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, encoder_out=None, causal_mask=None):
        # 1) self-attention enmascarado
        h, self_w = self.self_attn(self.ln1(x), attn_mask=causal_mask)
        x = x + self.dropout(h)

        # 2) cross-attention opcional (caso encoder-decoder)
        cross_w = None
        if encoder_out is not None:
            h, cross_w = self.cross_attn(self.ln_cross(x), x_kv=encoder_out, attn_mask=None)
            x = x + self.dropout(h)

        # 3) FFN
        x = x + self.ffn(self.ln2(x))
        return x, self_w, cross_w

# Pruebas de forma
B, Tsrc, Ttgt, D = 2, 7, 6, 64
enc_x = torch.randn(B, Tsrc, D)
dec_x = torch.randn(B, Ttgt, D)

enc_block = EncoderBlock(D, n_heads=4, d_ff=128)
dec_block = DecoderBlock(D, n_heads=4, d_ff=128, use_rope=True, use_alibi=False)

enc_out, enc_attn = enc_block(enc_x)
dec_out, self_w, cross_w = dec_block(dec_x, encoder_out=enc_out, causal_mask=make_causal_mask(Ttgt))

print("Salida del encoder:", enc_out.shape)
print("Salida del decoder:", dec_out.shape)
print("Self-attention del decoder:", self_w.shape)
print("Cross-attention del decoder:", cross_w.shape if cross_w is not None else None)


##### **Actividad guiada 5 - Encoder, decoder y cross-attention**

1. Compara **encoder-only** vs. **decoder-only** en términos de:
   - objetivo de entrenamiento,
   - tipo de máscara,
   - tipo de tarea.

2. Explica qué hace el **cross-attention** en una arquitectura encoder/decoder.
   ¿Qué parte produce Q y qué parte produce K/V?

3. Relaciona cada variante con tareas típicas:
   - clasificación/tagging
   - traducción
   - generación autoregresiva (LLM)

4. ¿Por qué los LLMs modernos suelen usar decoder-only aunque otras variantes existan?

##### **Respuesta del estudiante - Actividad 5**

- **Encoder-only vs. Decoder-only:**  
- **Cross-attention:**  
- **Tareas por variante:**  
- **Razón de uso en LLMs:**  


##### **5.1 Variante decoder-only (LLM) para next-token prediction**

Construimos un modelo pequeño autoregresivo (solo **decoder**) para ilustrar:

- embeddings de token
- estrategia posicional (sin/cos o learned)
- pila de bloques decoder
- proyección final a vocabulario


In [ ]:
class TinyDecoderOnlyLM(nn.Module):
    def __init__(
        self,
        vocab_size: int,
        d_model: int = 64,
        n_heads: int = 4,
        d_ff: int = 128,
        n_layers: int = 2,
        max_len: int = 128,
        dropout: float = 0.1,
        pos_mode: str = "sin",   # "sin" o "learned"
        use_rope: bool = True,
        use_alibi: bool = False,
    ):
        super().__init__()
        self.vocab_size = vocab_size
        self.d_model = d_model
        self.max_len = max_len
        self.pos_mode = pos_mode

        self.token_emb = nn.Embedding(vocab_size, d_model)
        if pos_mode == "sin":
            self.pos_enc = SinusoidalPositionalEncoding(d_model, max_len=max_len)
        elif pos_mode == "learned":
            self.pos_enc = LearnedPositionalEncoding(d_model, max_len=max_len)
        else:
            raise ValueError("pos_mode debe ser 'sin' o 'learned'")

        self.dropout = nn.Dropout(dropout)
        self.blocks = nn.ModuleList([
            DecoderBlock(d_model, n_heads, d_ff, dropout=dropout, use_rope=use_rope, use_alibi=use_alibi)
            for _ in range(n_layers)
        ])
        self.ln_f = nn.LayerNorm(d_model)
        self.head = nn.Linear(d_model, vocab_size, bias=False)

    def forward(self, idx: torch.Tensor):
        # idx: [B, T]
        B, T = idx.shape
        assert T <= self.max_len, "Secuencia demasiado larga para el max_len configurado"

        x = self.token_emb(idx)              # [B,T,D]
        x = self.pos_enc(x)                  # [B,T,D]
        x = self.dropout(x)

        causal = make_causal_mask(T, device=idx.device)
        last_attn = None
        for block in self.blocks:
            x, self_w, _ = block(x, encoder_out=None, causal_mask=causal)
            last_attn = self_w

        x = self.ln_f(x)
        logits = self.head(x)                # [B,T,V]
        return logits, last_attn

# Prueba rápida
model_test = TinyDecoderOnlyLM(vocab_size=vocab_size, pos_mode="sin", use_rope=True, use_alibi=False).to(device)
batch_ids = torch.tensor([
    vocab.encode("the llama runs fast"),
    vocab.encode("the dog barks loudly")
], dtype=torch.long, device=device)
logits, attn_last = model_test(batch_ids)
print("Lote de entrada:", batch_ids.shape)
print("Logits:", logits.shape)          # [B,T,V]
print("Última atención:", attn_last.shape if attn_last is not None else None)


##### **Actividad guiada 6 - Diseño del modelo decoder-only**

1. Identifica los componentes mínimos de un modelo autoregresivo tipo LLM en este cuaderno.

2. ¿Por qué el modelo aplica:
   - embedding de token,
   - estrategia posicional,
   - bloques decoder,
   - LayerNorm final,
   - proyección a vocabulario?

3. Explica qué representa la salida `logits` con forma `[B, T, V]`.
   ¿Qué dimensión se usa para predecir el siguiente token?

4. ¿Qué diferencia conceptual hay entre "tener logits" y "tener probabilidades"?

##### **Respuesta del estudiante - Actividad 6**

- **Componentes mínimos del decoder-only:**  
- **Rol de cada etapa:**  
- **Interpretación de logits [B,T,V]:**  
- **Logits vs. probabilidades:**  

#### **6.  Mini entrenamiento**

Usamos un `Dataset` y `DataLoader` personalizados. 

Usualmente el objetivo de entrenamiento:

- **Next-token prediction** (modelo autoregresivo)
- `CrossEntropyLoss` con `label_smoothing`
- `AdamW`
- Warmup + cosine decay
- Gradient clipping


In [ ]:
class NextTokenDataset(Dataset):
    def __init__(self, texts: List[str], vocab: SimpleVocab, block_size: int = 12):
        self.examples = []
        self.block_size = block_size
        for txt in texts:
            ids = vocab.encode(txt, add_bos=True, add_eos=True)
            # Ventanas deslizantes (corpus pequeño, basta 1-2 ventanas por línea)
            if len(ids) < 3:
                continue
            for start in range(0, max(1, len(ids) - 2), max(1, block_size // 2)):
                chunk = ids[start:start + block_size]
                if len(chunk) < 3:
                    continue
                self.examples.append(chunk)

        self.pad_id = vocab.pad_id

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, idx):
        ids = self.examples[idx]
        x = ids[:-1]   # tokens de entrada
        y = ids[1:]    # objetivos del siguiente token
        return torch.tensor(x, dtype=torch.long), torch.tensor(y, dtype=torch.long)

def collate_pad(batch, pad_id: int):
    xs, ys = zip(*batch)
    max_len = max(x.size(0) for x in xs)
    xb = torch.full((len(xs), max_len), pad_id, dtype=torch.long)
    yb = torch.full((len(ys), max_len), pad_id, dtype=torch.long)
    for i, (x, y) in enumerate(zip(xs, ys)):
        xb[i, :x.size(0)] = x
        yb[i, :y.size(0)] = y
    return xb, yb

dataset = NextTokenDataset(corpus, vocab, block_size=10)
loader = DataLoader(dataset, batch_size=4, shuffle=True,
                    collate_fn=lambda b: collate_pad(b, vocab.pad_id))

xb, yb = next(iter(loader))
print("Tamaño del dataset:", len(dataset))
print("Lote x:", xb.shape, "Lote y:", yb.shape)
print("Ejemplo de ids x:", xb[0].tolist())
print("Ejemplo de ids y:", yb[0].tolist())


In [ ]:
# Configuración del modelo
train_model = TinyDecoderOnlyLM(
    vocab_size=vocab_size,
    d_model=96,
    n_heads=4,
    d_ff=192,
    n_layers=2,
    max_len=32,
    dropout=0.1,
    pos_mode="sin",      # probar "learned"
    use_rope=True,       # RoPE dentro del self-attention
    use_alibi=False      # poner True para probar ALiBi
).to(device)

# Pérdida: ignorar padding y añadir label smoothing
criterion = nn.CrossEntropyLoss(ignore_index=vocab.pad_id, label_smoothing=0.1)

optimizer = torch.optim.AdamW(train_model.parameters(), lr=3e-3, weight_decay=0.01)

# Calentamiento + decaimiento coseno
num_epochs = 30
steps_per_epoch = len(loader)
total_steps = num_epochs * steps_per_epoch
warmup_steps = max(5, int(0.1 * total_steps))

def lr_lambda(step: int):
    if step < warmup_steps:
        return float(step + 1) / float(warmup_steps)
    progress = (step - warmup_steps) / max(1, total_steps - warmup_steps)
    return 0.5 * (1.0 + math.cos(math.pi * progress))

scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lr_lambda)

print(f"Pasos totales={total_steps}, pasos de calentamiento={warmup_steps}")


In [ ]:
# Bucle de entrenamiento (demostración pequeña)
train_model.train()
loss_history = []
lr_history = []

global_step = 0
for epoch in range(1, num_epochs + 1):
    epoch_loss = 0.0
    tokens_count = 0

    for xb, yb in loader:
        xb = xb.to(device)
        yb = yb.to(device)

        optimizer.zero_grad(set_to_none=True)
        logits, _ = train_model(xb)  # [B,T,V]

        # Aplanar para CE
        B, T, V = logits.shape
        loss = criterion(logits.view(B * T, V), yb.view(B * T))

        loss.backward()
        # Recorte de gradiente para estabilizar el entrenamiento
        nn.utils.clip_grad_norm_(train_model.parameters(), max_norm=1.0)

        optimizer.step()
        scheduler.step()

        global_step += 1
        epoch_loss += loss.item()
        lr_history.append(optimizer.param_groups[0]["lr"])
        loss_history.append(loss.item())

        # Contar objetivos válidos (no-pad) para un reporte aproximado
        tokens_count += (yb != vocab.pad_id).sum().item()

    if epoch % 5 == 0 or epoch == 1:
        avg_loss = epoch_loss / len(loader)
        print(f"Epoca {epoch:02d} | avg_loss={avg_loss:.4f} | lr={optimizer.param_groups[0]['lr']:.6f}")

plt.figure(figsize=(6, 3))
plt.plot(loss_history)
plt.title("Pérdida de entrenamiento")
plt.xlabel("Step")
plt.ylabel("Entropía cruzada")
plt.show()

plt.figure(figsize=(6, 3))
plt.plot(lr_history)
plt.title("Tasa de aprendizaje (calentamiento + coseno)")
plt.xlabel("Step")
plt.ylabel("LR")
plt.show()


##### **Actividad guiada 7 - Entrenamiento y optimización**

1. Explica el propósito de cada elemento del entrenamiento:
   - `CrossEntropyLoss`
   - `label_smoothing`
   - `AdamW`
   - warmup
   - cosine decay
   - gradient clipping

2. ¿Por qué en `CrossEntropyLoss` se ignoran posiciones con `<pad>`?

3. Interpreta las curvas de:
   - pérdida de entrenamiento,
   - tasa de aprendizaje.

   ¿Qué comportamiento esperarías al inicio (warmup) y después?

4. Si el entrenamiento fuera inestable (NaNs o pérdida explosiva), ¿qué revisarías primero y por qué?

##### **Respuesta del estudiante - Actividad 7**

- **Rol de los componentes de optimización:**  
- **Padding e ignore_index:**  
- **Interpretación de curvas:**  
- **Plan de diagnóstico de inestabilidad:**  

In [ ]:
# Extensión opcional (estudiante)
# Ejecuta 2 experimentos y compara resultados:
# 1) pos_mode="sin" vs. pos_mode="learned"
# 2) use_rope=True vs. use_rope=False
# 3) use_alibi=True vs. use_alibi=False
#
# Reporta luego (en texto):
# - pérdida final aproximada
# - estabilidad
# - observaciones

#### **7. Decoding: temperatura, top-k y top-p (nucleus)**

Aunque el foco del cuaderno es el **bloque Transformer**, añadimos una celda de generación para conectar con el flujo de LLM:

- **temperatura**: ajusta la entropía de la distribución
- **top-k**: limita a los k tokens más probables
- **top-p**: limita al menor conjunto con masa acumulada >= p


In [ ]:
@torch.no_grad()
def sample_next_token(logits_1d, temperature=1.0, top_k=None, top_p=None):
    logits = logits_1d / max(1e-6, temperature)

    if top_k is not None:
        k = min(top_k, logits.size(-1))
        values, _ = torch.topk(logits, k=k)
        thresh = values[..., -1].unsqueeze(-1)
        logits = torch.where(logits < thresh, torch.full_like(logits, float("-inf")), logits)

    if top_p is not None:
        sorted_logits, sorted_idx = torch.sort(logits, descending=True)
        probs = F.softmax(sorted_logits, dim=-1)
        cumprobs = torch.cumsum(probs, dim=-1)
        # Eliminar tokens después del umbral
        sorted_mask = cumprobs > top_p
        sorted_mask[..., 1:] = sorted_mask[..., :-1].clone()
        sorted_mask[..., 0] = False
        sorted_logits[sorted_mask] = float("-inf")
        # Reubicar en el orden original
        logits = torch.full_like(logits, float("-inf"))
        logits.scatter_(dim=-1, index=sorted_idx, src=sorted_logits)

    probs = F.softmax(logits, dim=-1)
    return torch.multinomial(probs, num_samples=1)

@torch.no_grad()
def generate_text(model, prompt: str, max_new_tokens=12, temperature=1.0, top_k=None, top_p=None):
    model.eval()
    ids = vocab.encode(prompt, add_bos=True, add_eos=False)
    idx = torch.tensor(ids, dtype=torch.long, device=device).unsqueeze(0)

    for _ in range(max_new_tokens):
        if idx.size(1) > model.max_len:
            idx_cond = idx[:, -model.max_len:]
        else:
            idx_cond = idx

        logits, _ = model(idx_cond)
        next_logits = logits[0, -1]
        next_id = sample_next_token(next_logits, temperature=temperature, top_k=top_k, top_p=top_p)
        idx = torch.cat([idx, next_id.view(1, 1)], dim=1)

        if next_id.item() == vocab.eos_id:
            break

    return vocab.decode(idx[0].tolist())

for cfg in [
    {"temperature": 1.0, "top_k": None, "top_p": None},
    {"temperature": 0.8, "top_k": 5, "top_p": None},
    {"temperature": 1.0, "top_k": None, "top_p": 0.9},
]:
    out = generate_text(train_model, prompt="the llama", max_new_tokens=8, **cfg)
    print(cfg, "->", out)


##### **Actividad guiada 8 - Decodificación y control de generación**

1. Explica el efecto de la **temperatura** sobre la distribución de probabilidad.

2. Compara **top-k** vs. **top-p**:
   - ¿qué restringe cada uno?
   - ¿cuándo uno puede ser más flexible que el otro?

3. ¿Por qué una estrategia de muestreo puede producir resultados diferentes aun con el mismo prompt?

4. Propón una configuración razonable (temperatura/top-k/top-p) para:
   - salida más conservadora
   - salida más diversa

##### **Respuesta del estudiante - Actividad 8**

- **Temperatura:**  
- **Top-k vs. Top-p:**  
- **Variabilidad de la generación:**  
- **Configuraciones propuestas:**  

#### **8. Variante encoder-only (MLM simplificado, conceptual)**

Para cerrar, mostramos un esqueleto *encoder-only* (estilo BERT) que produce representaciones contextualizadas y una cabecera de clasificación por token (útil para **MLM**, tagging, etc.).  
No entrenaremos MLM completo aquí; el objetivo es mostrar **la diferencia estructural** respecto al decoder-only.


In [ ]:
class TinyEncoderOnly(nn.Module):
    def __init__(self, vocab_size, d_model=64, n_heads=4, d_ff=128, n_layers=2, max_len=64, dropout=0.1):
        super().__init__()
        self.token_emb = nn.Embedding(vocab_size, d_model)
        self.pos_enc = LearnedPositionalEncoding(d_model, max_len=max_len)  # encoder-only suele usar posiciones absolutas aprendidas
        self.blocks = nn.ModuleList([EncoderBlock(d_model, n_heads, d_ff, dropout=dropout) for _ in range(n_layers)])
        self.ln_f = nn.LayerNorm(d_model)
        self.token_head = nn.Linear(d_model, vocab_size)  # cabecera por token (por ejemplo, MLM)

    def forward(self, idx):
        x = self.token_emb(idx)
        x = self.pos_enc(x)
        attn_last = None
        for blk in self.blocks:
            x, attn_last = blk(x)
        x = self.ln_f(x)
        logits = self.token_head(x)
        return logits, attn_last

enc_model = TinyEncoderOnly(vocab_size=vocab_size).to(device)
enc_logits, enc_attn = enc_model(batch_ids)
print("Logits encoder-only:", enc_logits.shape)
print("Atención encoder-only:", enc_attn.shape)


##### **Actividad guiada 9 - Comparación final de paradigmas**

Elabora una síntesis final (máximo 12 líneas) que compare:

- **Encoder-only**
- **Decoder-only**
- **Encoder/Decoder**

Tu síntesis debe incluir:
1. tipo de atención / máscara,
2. objetivo de entrenamiento típico,
3. tareas frecuentes,
4. una ventaja y una limitación de cada enfoque.

> Sugerencia: puedes responder en formato tabla.

##### **Respuesta del estudiante - Actividad 9**

| Variante | Atención / máscara | Objetivo típico | Tareas | Ventaja | Limitación |
|---|---|---|---|---|---|
| Encoder-only |  |  |  |  |  |
| Decoder-only |  |  |  |  |  |
| Encoder/Decoder |  |  |  |  |  |

#### **Ejercicios**

#### **1.  Visualizar atención con y sin máscara causal**

Compara cómo cambia el mapa de atención cuando se activa/desactiva la máscara causal.

**Instrucciones**

1. Reutiliza `scaled_dot_product_attention`, `make_causal_mask` y tensores `q, k, v` (o créalos de nuevo).
2. Calcula atención:

   * **sin máscara**
   * **con máscara causal**
3. Grafica ambas matrices de atención para la **cabecera 0**.
4. Escribe una observación breve en comentario dentro del código.

**Qué debe observarse**

* Sin máscara, cada posición puede atender a toda la secuencia.
* Con máscara causal, las posiciones futuras quedan bloqueadas.

#### **2. Comparar codificación posicional sinusoidal vs. aprendida**

Observa diferencias en forma y activación entre dos estrategias de posición.

**Instrucciones**

1. Crea un tensor `x` de ceros con forma `[1, T, D]`.
2. Aplícale:

   * `SinusoidalPositionalEncoding`
   * `LearnedPositionalEncoding`
3. Grafica ambos resultados con `imshow`.
4. Calcula una métrica sencilla de comparación (por ejemplo, norma o media).

**Sugerencia**

* Usa `T=64`, `D=32`.
* Compara visualmente patrones regulares (sinusoidal) vs. patrones aprendidos.

#### **3. Activar RoPE y ALiBi en el bloque de atención**

Ejecuta el mismo bloque con distintas configuraciones posicionales modernas.

**Instrucciones**

1. Crea un lote aleatorio `x_demo` de forma `[B, T, D]`.
2. Ejecuta tres variantes del bloque:

   * **base** (`use_rope=False`, `use_alibi=False`)
   * **con RoPE**
   * **con ALiBi**
3. Imprime formas de salida y atención.
4. Grafica la atención de la **cabecera 0** para cada variante.

**Qué comparar**

* Forma del mapa de atención.
* Cambios visuales entre variantes.
* Si se mantiene la forma `[B,H,T,T]`.

#### **4. Mini experimento de entrenamiento (sinusoidal vs learned)**

Compara la pérdida de entrenamiento con dos estrategias posicionales.

**Instrucciones**

1. Reutiliza `NextTokenDataset`, `DataLoader` y `TinyDecoderOnlyLM`.
2. Entrena **dos modelos pequeños** (5-10 épocas):

   * `pos_mode="sin"`
   * `pos_mode="learned"`
3. Guarda la pérdida promedio por época.
4. Grafica ambas curvas de pérdida.
5. Escribe una conclusión breve en comentario.

**Qué reportar**

* Pérdida final aproximada de cada variante.
* Estabilidad observada.
* Si alguna converge más rápido en este corpus pequeño.

#### **5. Generación controlada (temperatura, top-k, top-p)**

Compara el efecto de los hiperparámetros de decodificación.

**Instrucciones**

1. Reutiliza `generate_text`.
2. Usa un mismo prompt (por ejemplo: `"the llama"` o `"transformers use"`).
3. Genera texto con al menos **4 configuraciones**:

   * base
   * baja temperatura
   * top-k
   * top-p
4. Imprime resultados y comenta cuál configuración produce salidas más conservadoras o más diversas.

**Sugerencia de configuraciones**

* `temperature=1.0`
* `temperature=0.7`
* `top_k=5`
* `top_p=0.9`

**Qué analizar**

* Coherencia
* Diversidad
* Repetición
* Sensibilidad a la aleatoriedad


In [ ]:
## Tus respuestas

#### **Resumen final**

- El **bloque Transformer** combina:
  - **Self-attention** (Q, K, V)
  - **Multi-head attention**
  - **Residual + LayerNorm**
  - **FFN**
- La complejidad de la atención densa escala como **$O(T^2)$** por la matriz de logits **T×T**.
- **RoPE** y **ALiBi** son variantes modernas para codificar posición directamente en la atención.
- En práctica:
  - **encoder-only** -> representaciones/contexto global (BERT-like)
  - **decoder-only** -> predicción autoregresiva (LLM)
  - **encoder/decoder** -> condicionamiento por cross-attention (traducción, T5-like)

